In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

In [ ]:
# Generating synthetic data
X, y = make_blobs(n_samples=200, centers=2, n_features=2, cluster_std=0.8, random_state=42)

np.random.seed(42)

idx_class_0 = np.where(y == 0)[0]
idx_class_1 = np.where(y == 1)[0]

noisy_indices_0_to_1 = np.random.choice(idx_class_0, 5, replace=False)
y[noisy_indices_0_to_1] = 1

noisy_indices_1_to_0 = np.random.choice(idx_class_1, 20, replace=False)
y[noisy_indices_1_to_0] = 0

In [ ]:
# Scaling featurs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Split into training and validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size = 0.3, random_state = 42)

print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Validation data shape: {X_val.shape}, {y_val.shape}")

plt.figure(figsize=(8, 6))
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='viridis', s=50, alpha=0.7, label='Training Data')
plt.scatter(X_val[:, 0], X_val[:, 1], c=y_val, cmap='viridis', marker='x', s=100, alpha=0.7, label='Validation Data')
plt.title('Linearly Separable Data with Misclassified Points')
plt.xlabel('Feature 1 (Scaled)')
plt.ylabel('Feature 2 (Scaled)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Neural network without dropout
def build_simple_model():
  model = keras.Sequential([
      layers.Dense(64, activation = 'relu', input_shape = (2,)),
      layers.Dense(64, activation = 'relu'),
      layers.Dense(64, activation = 'relu'),
      layers.Dense(32, activation = 'relu'),
      layers.Dense(1, activation = 'sigmoid')
  ])
  model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

model_without_dropout = build_simple_model()
model_without_dropout.summary()

In [ ]:
# Training the model
history_without_dropout = model_without_dropout.fit(X_train, y_train,
                                                    epochs = 200,
                                                    batch_size = 32,
                                                    validation_data = (X_val, y_val),
                                                    verbose = 0)
# plotting loss and accuracy
plt.figure(figsize = (10, 6))

plt.subplot(1, 2, 2)
plt.plot(history_without_dropout.history['loss'], label = 'Training Loss')
plt.plot(history_without_dropout.history['val_loss'], label = 'Validation Loss')
plt.title('Model Training Without Dropout')
plt.xlabel('Epoch')
plt.ylabel('binary Crossentropy')
plt.legend()
plt.grid(True)
plt.show()

plt.subplot(1, 2, 2)
plt.plot(history_without_dropout.history['accuracy'], label='Training Accuracy')
plt.plot(history_without_dropout.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy without Dropout (Overfitting)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# decision boundary
def plot_decision_boundary(model, X, y, title, ax):
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    Z = (Z > 0.5).astype(int)

    ax.contourf(xx, yy, Z, alpha=0.4, cmap='coolwarm')
    ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='viridis', edgecolors='k', alpha=0.7)
    ax.set_title(title)
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')


fig, ax = plt.subplots(figsize=(8, 6))
plot_decision_boundary(model_without_dropout, X_scaled, y, 'Decision Boundary: No Dropout (Overfit)', ax)
plt.show()

In [ ]:
# Dropout
class CustomDropout(layers.Layer):
  def __init__(self, rate, **kwargs):
    super(CustomDropout, self).__init__(**kwargs)
    if not 0 <= rate <= 1:
      raise ValueError(f"Dropout rate must be between 0 annd 1. Got: {rate}")
    self.rate = rate

  def call(self, inputs, training = None):
    if training:
      keep_prob = 1.0 - self.rate
      mask = tf.cast(tf.random.uniform(tf.shape(inputs)) > self.rate, dtype = tf.float32)
      outputs = inputs * mask / keep_prob
    else:
      outputs = inputs
    return outputs

  def get_config(self):
    config = super(CustomerDropout, self).get.config()
    config.update({"rate": self.rate})
    return config



In [ ]:
# Neural network with dropout
def new_model(dropout_rate = 0.3):
  model = keras.Sequential([
      layers.Dense(64, activation = 'relu', input_shape = (2,)),
      CustomDropout(dropout_rate),
      layers.Dense(64, activation = 'relu'),
      CustomDropout(dropout_rate),
      layers.Dense(64, activation = 'relu'),
      CustomDropout(dropout_rate),
      layers.Dense(32, activation = 'relu'),
      CustomDropout(dropout_rate),
      layers.Dense(1, activation = 'sigmoid')
  ])
  model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

model_with_dropout = new_model(dropout_rate = 0.3)
model_with_dropout.summary()

In [ ]:
# Training the new Model
history_with_dropout = model_with_dropout.fit(X_train, y_train,
                                     epochs = 200,
                                     batch_size = 32,
                                     validation_data = (X_val, y_val),
                                     verbose = 0)

# plotting loss and accuracy
plt.figure(figsize = (10,6))

plt.subplot(1, 2, 1)
plt.plot(history_with_dropout.history['loss'], label = 'Training Loss (with dropout)')
plt.plot(history_with_dropout.history['val_loss'], label = 'Validatiion Loss (with dropout)')
plt.title('Model Trainin With Custom Dropout')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error(MSE)')
plt.legend()
plt.grid(True)
plt.show()

plt.subplot(1, 2, 2)
plt.plot(history_with_dropout.history['accuracy'], label='Training Accuracy (with Dropout)')
plt.plot(history_with_dropout.history['val_accuracy'], label='Validation Accuracy (with Dropout)')
plt.title('Accuracy with Custom Dropout (Improved Generalization)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# decision boundary
def plot_decision_boundary(model, X, y, title, ax):
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    Z = (Z > 0.5).astype(int)

    ax.contourf(xx, yy, Z, alpha=0.4, cmap='coolwarm')
    ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='viridis', edgecolors='k', alpha=0.7)
    ax.set_title(title)
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')
fig, ax = plt.subplots(figsize=(8, 6))
plot_decision_boundary(model_with_dropout, X_scaled, y, 'Decision Boundary: With Dropout (Smoother)', ax)
plt.show()

In [ ]:
# Comparision
plt.figure(figsize = (12,7))
plt.plot(history_without_dropout.history['val_loss'], label = 'Validation Loss (without Dropout)', linestyle = '--')
plt.plot(history_with_dropout.history['val_loss'], label = 'Validation Loss (with Dropout)')
plt.title('Comparision of Validation Loss: No Dropout vs. With Dropout')
plt.xlabel('Epochs')
plt.ylabel('Binary Crossentropy')
plt.legend()
plt.grid(True)
plt.show()